In [11]:
import pandas as pd

df = pd.read_csv("../data/raw/data.csv")
df = df.drop_duplicates()
df = df.dropna(subset=["Customer_ID", "Amount", "Product_Category"])
df = df.fillna({"Income": "Unknown", "Feedback": "No Feedback"})
df['Date'] = pd.to_datetime(df['Date'])


In [12]:
from sklearn.preprocessing import LabelEncoder

cat_cols = ["Gender", "Income", "Customer_Segment", "Product_Category"]
for col in cat_cols:
    df[col] = LabelEncoder().fit_transform(df[col].astype(str))


In [13]:
from prophet import Prophet

df['YearMonth'] = df['Date'].dt.to_period('M')
sales = df.groupby(['YearMonth', 'Product_Category'])['Amount'].sum().reset_index()
sales['YearMonth'] = sales['YearMonth'].dt.to_timestamp()
cat_sales = sales[sales['Product_Category'] == 2][['YearMonth', 'Amount']]  # choose a popular category
model = Prophet()

model.fit(cat_sales)
future = model.make_future_dataframe(periods=3, freq='M')
forecast = model.predict(future)
print(forecast[['ds', 'yhat']].tail())


22:26:43 - cmdstanpy - INFO - Chain [1] start processing
22:26:44 - cmdstanpy - INFO - Chain [1] done processing


           ds          yhat
10 2024-01-01  1.493495e+06
11 2024-02-01  1.489739e+06
12 2024-02-29  1.486347e+06
13 2024-03-31  1.482591e+06
14 2024-04-30  1.478956e+06


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/prophet/forecaster.py:1872: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(


In [14]:
X = df[["Age","Gender","Income","Customer_Segment","Amount","Total_Purchases"]]
y = df["Customer_Segment"]
from sklearn.impute import SimpleImputer
num_cols = ["Age","Amount","Total_Purchases"]
X[num_cols] = SimpleImputer(strategy="median").fit_transform(X[num_cols])


/var/folders/vf/rbbd31d52x71bpbg3sn62p3r0000gn/T/ipykernel_48267/47305341.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[num_cols] = SimpleImputer(strategy="median").fit_transform(X[num_cols])


In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)


In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

models = [
    LogisticRegression(max_iter=500, random_state=42),
    RandomForestClassifier(n_estimators=100, random_state=42),
    GradientBoostingClassifier(random_state=42),
    SVC(kernel="rbf", probability=True, random_state=42),
    KNeighborsClassifier(n_neighbors=5)
]

for m in models:
    m.fit(X_train, y_train)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 500 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=500).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from sklearn.metrics import accuracy_score

for m in models:
    y_pred = m.predict(X_test)
    print(type(m).__name__, "Accuracy:", accuracy_score(y_test, y_pred))
